# Pre-Processing Image

## import library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

import keras
from keras.preprocessing.image import ImageDataGenerator

## Load data from GDrive

In [ ]:
train_dir = '../input/tomato-disease-ver2/tomato_disease_ready ver2/train'
val_dir = '../input/tomato-disease-ver2/tomato_disease_ready ver2/validation'
test_dir = '../input/tomato-disease-ver2/tomato_disease_ready ver2/test'

## Menghitung jumlah dataset

In [ ]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count = 0 
  for current_path, dirs, files in os.walk(directory):
    for dr in dirs:
      count += len(glob.glob(os.path.join(current_path, dr+"/*")))
  return count

In [ ]:
data_training = get_files(train_dir)
data_validation = get_files(val_dir)
data_testing = get_files(test_dir)
num_classes = len(glob.glob(train_dir + "/*"))

print('Jumlah data training :', data_training)
print('Jumlah data validation :', data_validation)
print('Jumlah data testing :', data_testing)
print('Jumlah Kelas Dataset :', num_classes)

## Pre-Processing data input

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   horizontal_flip=True,
                                   zoom_range = 0.3)

val_datagen = ImageDataGenerator(rescale=1/255,
                                 horizontal_flip=True,
                                 zoom_range = 0.3)

test_datagen = ImageDataGenerator(rescale=1/255,
                                  horizontal_flip=False,
                                  vertical_flip=False)

## Mengatur Ukuran dari image/citra input

In [ ]:
image_width = 400
image_height = 400

mode = 'rgb'

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(image_width, image_height),
                                                    batch_size=32,
                                                    color_mode = mode)

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size = (image_width, image_height),
                                                batch_size=32,
                                                color_mode = mode)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size = (image_width, image_height),
                                                  batch_size=32,
                                                  color_mode = mode)

# Build CNN Architecure

In [ ]:
train_generator.class_indices

In [ ]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import plot_model

from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, BatchNormalization

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(image_width, image_height, 3), padding='same', strides=1, activation='relu', name='konvolusi_1'))
#model.add(Conv2D(64, (3,3), strides=1, activation='relu', name='konvolusi_1b'))
model.add(MaxPooling2D(pool_size = (2, 2), name='pool_layer_1'))


model.add(Conv2D(64, (3,3), padding='same', strides=1, activation='relu', name='konvolusi_2'))
#model.add(Conv2D(256, (3,3), strides=1, activation='relu', name='konvolusi_2b'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool_layer_2'))

model.add(Conv2D(128, (3,3), padding='same', strides=1, activation='relu', name='konvolusi_3'))
#model.add(Conv2D(512, (3,3), strides=1, activation='relu', name='konvolusi_3b'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool_layer_3'))

model.add(Conv2D(256, (3,3), padding='same', strides=1, activation='relu', name='konvolusi_4'))
#model.add(Conv2D(256, (3,3), strides=1, activation='relu', name='konvolusi_4b'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool_layer_4'))

model.add(Conv2D(512, (3,3), padding='same', strides=1, activation='relu', name='konvolusi_5'))
#model.add(Conv2D(512, (3,3), strides=1, activation='relu', name='konvolusi_5b'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool_layer_5'))

model.add(Flatten())
model.add(Dense(1024, activation='relu', name='ANN_1'))
model.add(Dense(1024, activation='relu', name='ANN_1b'))
model.add(Dense(num_classes, activation='softmax', name='Output'))

model.summary()

In [ ]:
plot_model(model, show_layer_names=True, show_shapes=True, to_file='ini_model.png')

## Visualisasi Layer Konvolusi

In [ ]:
from keras.preprocessing import image
import numpy as np

img1 = image.load_img('../input/tomato-disease-ver2/tomato_disease_ready ver2/validation/healthy/000146ff-92a4-4db6-90ad-8fce2ae4fddd___GH_HL Leaf 259.1 - Copy.JPG')
plt.imshow(img1)

img1 = image.load_img('../input/tomato-disease-ver2/tomato_disease_ready ver2/validation/healthy/000146ff-92a4-4db6-90ad-8fce2ae4fddd___GH_HL Leaf 259.1 - Copy.JPG', target_size=(image_width, image_height, 1), color_mode='rgb')
img = image.img_to_array(img1)
img = img/255
img = np.expand_dims(img, axis=0)

In [ ]:
import matplotlib.image as mpig
from keras.models import  Model

conv_output = Model(inputs=model.input, outputs=model.get_layer('konvolusi_3').output)
conv_features = conv_output.predict(img)

fig = plt.figure(figsize= (28, 14))
columns = 8
rows = 4

for i in range (columns*rows):
  fig.add_subplot(rows, columns, i+1)
  plt.axis('off')
  plt.title('filter ke-' + str(i))
  plt.imshow(conv_features[0, :, :, i])
plt.show

# Training CNN Architecture

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 32

train = model.fit_generator(train_generator,
                            epochs=10,
                            steps_per_epoch = train_generator.samples // batch_size,
                            validation_data = val_generator,
                            validation_steps = val_generator.samples // batch_size,
                            verbose = 1)

## Plot Training Process

In [ ]:
acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']

epochs = range(1, len(acc) + 1)

# Plot akurasi data taraining dan validation
plt.plot(epochs, acc, 'b', label='Akurasi training')
plt.plot(epochs, val_acc, 'r', label='Akurasi validasi')
plt.title('Akurasi Training dan Validasi')
plt.legend()

plt.figure()

# Plot loss taraining dan validation
plt.plot(epochs, loss, 'b', label='Loss training')
plt.plot(epochs, val_loss, 'r', label='Loss validasi')
plt.title('Akurasi Training dan Validasi')
plt.legend()

plt.show()

In [ ]:
score, accuracy = model.evaluate(test_generator, verbose=1)
print("Score Testing adalah {}".format(score))
print("Akurasi Testing adalah {}".format(accuracy))

## Menyimpan Model 

In [ ]:
model.save('./model_saya_2.h5')

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(image_width,image_height),
                                                  batch_size=batch_size,
                                                  color_mode='rgb',
                                                  shuffle=False)

resolt = model.predict_classes(test_generator, batch_size=None, verbose=1)

In [ ]:
resolt

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt


data = {'y_Actual':    test_generator.classes,
        'y_Predicted': resolt
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

sn.heatmap(confusion_matrix, annot=True, fmt='.0f')
plt.show()

## Test data baru

In [ ]:
# Loading model
from keras.models import load_model
model = load_model('./model_saya_2.h5')

Classes = ['bacterial_spot',
           'early_blight',
           'healthy',
           'late_blight',
           'leaf_mold',
           'septoria_leaf_spot',
           'spotted_spider_mite',
           'target_spot',
           'yellow_leaf_curl_virus']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# pre-processing test data sama dengan train data
img_width = 400
img_height = 400
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

from keras.preprocessing import image
def prepare(img_path):
    img = image.load_img(img_path, target_size=(384,384,3))
    x = image.img_to_array(img)
    x = x/255
    return np.expand_dims(x, axis=0)

input_data = '../input/tomato-disease-ver2/tomato_disease_ready ver2/test/yellow_leaf_curl_virus/a2737028-9b65-4d23-900d-42f9f795f465___YLCV_GCREC 2489.JPG'
result = model.predict_classes([prepare(input_data)])
disease = image.load_img(input_data)
plt.imshow(disease)
print ('Terdeteksi penyakit : ', Classes[int(result)])
